# CS 5228


In [1]:
# All Imports
from utils import *
import pandas as pd
import locale
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
%matplotlib inline

# model training
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

# model evaluation
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

# classifiers
from sklearn.naive_bayes import GaussianNB # naive bayes
from sklearn.neighbors import KNeighborsClassifier # KNN
from sklearn.linear_model import LogisticRegression # logistic regression
from sklearn.tree import DecisionTreeClassifier # decision Tree
from sklearn.ensemble import RandomForestClassifier 
from sklearn.ensemble import GradientBoostingClassifier

# ignore warnings
import warnings
warnings.filterwarnings('ignore')
locale.setlocale(locale.LC_ALL,'')
pd.set_option('display.max_columns', None)

In [16]:
# Process Training Data
df_train = pd.read_csv("dataset/Xtrain.csv",dtype= {'Zip': 'object', 'NAICS':'object', 'NewExist':'object', 'FranchiseCode':'object', 'UrbanRural':'object'}, parse_dates=['ApprovalDate','DisbursementDate'])
y_train = pd.read_csv("dataset/Ytrain.csv")
df_train = pd.concat([df_train, y_train['ChargeOff']], axis = 1)

print("Training data size before removing duplicate:", df_train.shape)
df_train = df_train.drop_duplicates(subset=df_train.columns.difference(['Id']))
print("Training data size after removing duplicate:", df_train.shape)

df1 = data_preprocessing(df_train)
df2 = df1.dropna()
df3 = df1.fillna(-1)

base_dropna = one_hot_encoding_common(df2)
base_fillna = one_hot_encoding_common(df3)
feature_dropna = feature_transformation(base_dropna)
feature_fillna = feature_transformation(base_fillna)

Training data size before removing duplicate: (50000, 25)
Training data size after removing duplicate: (50000, 25)


In [32]:
# Process Test Data
df_test = pd.read_csv("dataset/Xtest.csv",dtype= {'Zip': 'object', 'NAICS':'object', 'NewExist':'object', 'FranchiseCode':'object', 'UrbanRural':'object'}, parse_dates=['ApprovalDate','DisbursementDate'])
df_test = data_preprocessing(df_test)
df_test.loc[(df_test['DisbursementDate'].isna()), 'DisbursementDate'] = pd.Timestamp(2017, 1, 1)
df_test.loc[(df_test['ApprovalDate'].isna()), 'ApprovalDate'] = pd.Timestamp(1900, 1, 1)
df_test = df_test.fillna(-1)
df_test = one_hot_encoding_common(df_test)

In [33]:
df_test.describe(include='all')

,Name,City,State,Zip,Bank,BankState,NAICS,ApprovalDate,ApprovalFY,Term,NoEmp,CreateJob,RetainedJob,FranchiseCode,DisbursementDate,DisbursementGross,GrAppv,SBA_Appv,NewExist_-1,NewExist_1,NewExist_2,UrbanRural_0,UrbanRural_1,UrbanRural_2,RevLineCr_N,RevLineCr_Y,LowDoc_-1,LowDoc_N,LowDoc_Y
count,100000,100000,100000,100000.000000,100000,100000,100000.000000,100000,100000.000000,100000.000000,100000.000000,100000.00000,100000.000000,100000.000000,100000,1.000000e+05,1.000000e+05,1.000000e+05,100000.000000,100000.000000,100000.000000,100000.000000,100000.00000,100000.00000,100000.000000,100000.000000,100000.000000,100000.00000,100000.000000
unique,96410,13109,52,NaN,3547,53,NaN,7907,NaN,NaN,NaN,NaN,NaN,NaN,3675,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,SUBWAY,LOS ANGELES,CA,NaN,BANK OF AMERICA NATL ASSOC,CA,NaN,2004-01-30 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,2006-06-30 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,98,1602,14869,NaN,11970,14406,NaN,125,NaN,NaN,NaN,NaN,NaN,NaN,1177,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
first,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1968-02-09 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,1968-03-19 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-05-12 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,2017-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,53789.025510,NaN,NaN,421790.956760,NaN,1968.201310,89.642570,9.912300,11.16305,13.676960,2595.417730,NaN,1.750579e+05,1.647051e+05,1.244969e+05,0.001160,0.708940,0.289900,0.275280,0.60408,0.12064,0.737600,0.262400,0.008100,0.89246,0.099440
std,NaN,NaN,NaN,30960.871596,NaN,NaN,250716.950871,NaN,258.721679,72.699021,70.572133,285.11886,285.472884,12362.965686,NaN,2.691642e+05,2.652480e+05,2.096262e+05,0.034039,0.454253,0.453718,0.446657,0.48905,0.32571,0.439941,0.439941,0.089635,0.30980,0.299253
min,NaN,NaN,NaN,0.000000,NaN,NaN,0.000000,NaN,-1.000000,0.000000,0.000000,0.00000,0.000000,0.000000,NaN,0.000000e+00,4.000000e+02,2.000000e+02,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000
25%,NaN,NaN,NaN,28273.000000,NaN,NaN,238220.000000,NaN,1998.000000,44.000000,2.000000,0.00000,0.000000,0.000000,NaN,3.500000e+04,2.500000e+04,1.600000e+04,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,1.00000,0.000000


In [4]:
base_fillna.describe(include='all')

,Name,City,State,Zip,Bank,BankState,NAICS,ApprovalDate,ApprovalFY,Term,NoEmp,CreateJob,RetainedJob,FranchiseCode,DisbursementDate,DisbursementGross,GrAppv,SBA_Appv,ChargeOff,NewExist_-1,NewExist_1,NewExist_2,UrbanRural_0,UrbanRural_1,UrbanRural_2,RevLineCr_N,RevLineCr_Y,LowDoc_-1,LowDoc_N,LowDoc_Y
count,50000,50000,50000,50000.000000,50000,50000,50000.000000,50000,50000.000000,50000.000000,50000.000000,50000.00000,50000.000000,50000.000000,50000,5.000000e+04,5.000000e+04,5.000000e+04,50000.000000,50000.000000,50000.000000,50000.000000,50000.00000,50000.00000,50000.000000,50000.000000,50000.000000,50000.00000,50000.000000,50000.000000
unique,48757,9488,51,NaN,2784,53,NaN,6934,NaN,NaN,NaN,NaN,NaN,NaN,2434,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,SUBWAY,LOS ANGELES,CA,NaN,BANK OF AMERICA NATL ASSOC,CA,NaN,2004-01-30 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,2007-04-30 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,55,814,7400,NaN,5895,7257,NaN,53,NaN,NaN,NaN,NaN,NaN,NaN,591,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
first,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1969-04-28 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-04-14 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,53918.218620,NaN,NaN,423739.328920,NaN,2002.148600,89.856540,9.475280,9.98176,12.441680,2584.720840,NaN,1.740080e+05,1.640501e+05,1.242661e+05,0.496960,0.000720,0.712720,0.286560,0.27754,0.60118,0.121280,0.734220,0.265780,0.00704,0.894280,0.098680
std,NaN,NaN,NaN,31022.935383,NaN,NaN,251283.168494,NaN,5.980605,73.066175,59.702302,266.95799,267.466654,12332.648265,NaN,2.615771e+05,2.616304e+05,2.080550e+05,0.499996,0.026823,0.452498,0.452159,0.44779,0.48966,0.326456,0.441752,0.441752,0.08361,0.307482,0.298235
min,NaN,NaN,NaN,0.000000,NaN,NaN,0.000000,NaN,1969.000000,0.000000,0.000000,0.00000,0.000000,0.000000,NaN,0.000000e+00,2.000000e+02,1.000000e+02,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
25%,NaN,NaN,NaN,28255.000000,NaN,NaN,238220.000000,NaN,1999.000000,44.000000,2.000000,0.00000,0.000000,0.000000,NaN,3.500000e+04,2.500000e+04,1.566000e+04,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.00000,1.000000,0.000000


In [5]:
feature_dropna.head()

,Name,City,State,Zip,Bank,BankState,NAICS,ApprovalDate,ApprovalFY,CreateJob,RetainedJob,FranchiseCode,DisbursementDate,DisbursementGross,GrAppv,SBA_Appv,ChargeOff,NewExist_1,NewExist_2,UrbanRural_0,UrbanRural_1,UrbanRural_2,RevLineCr_N,RevLineCr_Y,LowDoc_N,LowDoc_Y,NoEmp_Micro,NoEmp_Small,NoEmp_Medium,NoEmp_Large,Term_Short,Term_Intermediate,Term_Long,Term_Extra Long
0,"NPE Realty, LLC",Pembroke Pines,FL,33027,"TD BANK, NATIONAL ASSOCIATION",DE,334516,2010-04-16,2010,2,0,0,2010-05-01,2000000.0,2000000.0,1500000.0,0,1,0,0,1,0,1,0,1,0,0,1,0,0,0,0,1,0
1,"KEVCO CONSTRUCTION, LLC",SANDY,UT,84094,ZIONS FIRST NATIONAL BANK,UT,236115,2010-01-30,2010,1,3,0,2010-02-01,17000.0,17000.0,17000.0,1,1,0,0,1,0,1,0,1,0,1,0,0,0,0,1,0,0
2,EAST L A SHOE REPAIR,LOS ANGELES,CA,90022,BANK OF AMERICA NATL ASSOC,NC,811430,2002-07-29,2002,1,1,0,2002-08-31,17000.0,17000.0,8500.0,0,1,0,0,1,0,0,1,1,0,1,0,0,0,0,0,1,0
3,"RUSSLER, DANIEL C, MD",LODI,WI,53555,ASSOCIATED BANK NATL ASSOC,WI,0,1994-06-10,1994,0,0,0,1994-07-31,137000.0,137000.0,123300.0,0,0,1,1,0,0,1,0,1,0,1,0,0,0,0,0,1,0
4,AFFORDABLE FAMILY DENISTRY LLC,LEBANON,MO,65536,COMMERCE BANK,MO,621210,2006-02-06,2006,7,2,0,2006-03-31,46665.0,25000.0,12500.0,0,1,0,0,1,0,1,0,1,0,1,0,0,0,0,0,1,0


In [6]:
base_fillna.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50000 entries, 0 to 49999
Data columns (total 30 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Name               50000 non-null  object        
 1   City               50000 non-null  object        
 2   State              50000 non-null  object        
 3   Zip                50000 non-null  int32         
 4   Bank               50000 non-null  object        
 5   BankState          50000 non-null  object        
 6   NAICS              50000 non-null  int32         
 7   ApprovalDate       50000 non-null  datetime64[ns]
 8   ApprovalFY         50000 non-null  int16         
 9   Term               50000 non-null  int64         
 10  NoEmp              50000 non-null  int64         
 11  CreateJob          50000 non-null  int64         
 12  RetainedJob        50000 non-null  int64         
 13  FranchiseCode      50000 non-null  int32         
 14  Disbur

In [12]:
model_names = ['KNN', 'LR', 'DT', 'RF', 'GBM']
base_dropna_f1 = []
base_dropna_acc = []
base_fillna_f1 = []
base_fillna_acc = []
feature_dropna_f1 = []
feature_dropna_acc = []
feature_fillna_f1 = []
feature_fillna_acc = []

def calculate_acc_and_f1(classifier, x_train, y_train, x_test, y_test):
    classifier.fit(x_train, y_train)
    y_pred = classifier.predict(x_test)
    f1 = round(f1_score(y_test, y_pred, average='weighted') * 100, 2)
    acc = round(accuracy_score(y_test, y_pred) * 100, 2)
    return f1, acc

    
def train_single_classifier(classifier, df_in, f1_list, acc_list):
    df_x = df_in.drop(columns='ChargeOff')
    df_y = df_in['ChargeOff']
    x_train, x_test, y_train, y_test = train_test_split(df_x, df_y, test_size = 0.25, random_state=0)
    f1, acc = calculate_acc_and_f1(classifier, x_train, y_train, x_test, y_test)
    f1_list.append(f1)
    acc_list.append(acc)
    

def train_model(df_in, f1_list, acc_list):
    df_numeric = df_in.drop(columns=['Name','City','State','Bank','BankState'])
    df_numeric['ApprovalDate'] = (df_numeric['ApprovalDate'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')
    df_numeric['DisbursementDate'] = (df_numeric['DisbursementDate'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')
    train_single_classifier(KNeighborsClassifier(), df_numeric, f1_list, acc_list)
    train_single_classifier(LogisticRegression(), df_numeric, f1_list, acc_list)
    train_single_classifier(DecisionTreeClassifier(), df_numeric, f1_list, acc_list)
    train_single_classifier(RandomForestClassifier(), df_numeric, f1_list, acc_list)
    train_single_classifier(GradientBoostingClassifier(), df_numeric, f1_list, acc_list)
    

train_model(base_dropna, base_dropna_f1, base_dropna_acc)
# train_model(base_fillna, base_fillna_f1, base_fillna_acc)
train_model(feature_dropna, feature_dropna_f1, feature_dropna_acc)
# train_model(feature_fillna, feature_fillna_f1, feature_fillna_acc)

In [23]:
df_test.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 27 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   Name               99999 non-null   object        
 1   City               99999 non-null   object        
 2   State              99996 non-null   object        
 3   Zip                100000 non-null  int32         
 4   Bank               99889 non-null   object        
 5   BankState          99888 non-null   object        
 6   NAICS              100000 non-null  int32         
 7   ApprovalDate       100000 non-null  datetime64[ns]
 8   ApprovalFY         98304 non-null   float64       
 9   Term               100000 non-null  int64         
 10  NoEmp              100000 non-null  int64         
 11  CreateJob          100000 non-null  int64         
 12  RetainedJob        100000 non-null  int64         
 13  FranchiseCode      100000 non-null  int32    

In [38]:
model = GradientBoostingClassifier()
df_test1 = df_test.drop(columns=['Name','City','State','Bank','BankState', 'NewExist_-1', 'LowDoc_-1'])
df_test1['ApprovalDate'] = (df_test1['ApprovalDate'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')
df_test1['DisbursementDate'] = (df_test1['DisbursementDate'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')
base_dropna_x = base_dropna.drop(columns='ChargeOff')
base_dropna_x['ApprovalDate'] = (base_dropna_x['ApprovalDate'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')
base_dropna_x['DisbursementDate'] = (base_dropna_x['DisbursementDate'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')
base_dropna_y = base_dropna['ChargeOff']
base_dropna_x = base_dropna_x.drop(columns=['Name','City','State','Bank','BankState'])
model.fit(base_dropna_x, base_dropna_y)
test_pred = model.predict(df_test1)
pd.DataFrame(test_pred).to_csv('y_pred.csv')

In [35]:
base_dropna_x.dtypes

Zip                    int32
NAICS                  int32
ApprovalDate           int64
ApprovalFY             int16
Term                   int64
NoEmp                  int64
CreateJob              int64
RetainedJob            int64
FranchiseCode          int32
DisbursementDate       int64
DisbursementGross    float32
GrAppv               float32
SBA_Appv             float32
NewExist_1             uint8
NewExist_2             uint8
UrbanRural_0           uint8
UrbanRural_1           uint8
UrbanRural_2           uint8
RevLineCr_N            uint8
RevLineCr_Y            uint8
LowDoc_N               uint8
LowDoc_Y               uint8
dtype: object

In [36]:
df_test1.dtypes

Zip                    int32
NAICS                  int32
ApprovalDate           int64
ApprovalFY           float64
Term                   int64
NoEmp                  int64
CreateJob              int64
RetainedJob            int64
FranchiseCode          int32
DisbursementDate       int64
DisbursementGross    float32
GrAppv               float32
SBA_Appv             float32
NewExist_-1            uint8
NewExist_1             uint8
NewExist_2             uint8
UrbanRural_0           uint8
UrbanRural_1           uint8
UrbanRural_2           uint8
RevLineCr_N            uint8
RevLineCr_Y            uint8
LowDoc_-1              uint8
LowDoc_N               uint8
LowDoc_Y               uint8
dtype: object

In [13]:
# accuracy_record = pd.DataFrame({'Model': model_names, 'base_dropna_acc': base_dropna_acc, 'base_fillna_acc': base_fillna_acc, 'feature_dropna_acc': feature_dropna_acc, 'feature_fillna_acc': feature_fillna_acc})
accuracy_record = pd.DataFrame({'Model': model_names, 'base_dropna_acc': base_dropna_acc, 'feature_dropna_acc': feature_dropna_acc})
accuracy_record['acc_mean'] = accuracy_record.mean(axis=1).round(2)
accuracy_record.set_index('Model', inplace=True)
accuracy_record.loc['avg'] = accuracy_record.mean()

# F1_record = pd.DataFrame({'Model': model_names, 'base_dropna_f1': base_dropna_f1, 'base_fillna_f1': base_fillna_f1, 'feature_dropna_f1': feature_dropna_f1, 'feature_fillna_f1': feature_fillna_f1})
F1_record = pd.DataFrame({'Model': model_names, 'base_dropna_f1': base_dropna_f1, 'feature_dropna_f1': feature_dropna_f1})
F1_record['F1_mean'] = F1_record.mean(axis=1).round(2)
F1_record.set_index('Model', inplace=True)
F1_record.loc['avg'] = F1_record.mean()

print(accuracy_record)
print('\n')
print(F1_record)

       base_dropna_acc  feature_dropna_acc  acc_mean
Model                                               
KNN             68.200               68.20    68.200
LR              63.570               63.57    63.570
DT              88.030               70.87    79.450
RF              89.520               78.37    83.940
GBM             89.800               77.59    83.700
avg             79.824               71.72    75.772


       base_dropna_f1  feature_dropna_f1  F1_mean
Model                                            
KNN            68.200             68.200   68.200
LR             62.720             62.720   62.720
DT             88.030             70.870   79.450
RF             89.520             78.360   83.940
GBM            89.800             77.590   83.700
avg            79.654             71.548   75.602


In [14]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(["paris", "paris", "tokyo", "amsterdam"])

list(le.classes_)

le.transform(["tokyo", "tokyo", "paris"])

list(le.inverse_transform([2, 2, 1]))

['tokyo', 'tokyo', 'paris']